In [1]:
#This notebook was created with the help of ChatGPT
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import cross_val_score
import numpy as np


In [2]:
# Load dataset
filtered_df = pd.read_csv('filtered_mental_health_data.csv')

In [3]:
# Prepare dataset
feature_columns = [col for col in filtered_df.columns if not col.startswith('DSM')]
X = filtered_df[feature_columns]
y = filtered_df['DSM_MJD'].apply(lambda x: 1 if x == 1 else 0)

In [4]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [5]:
#hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

In [6]:
# Fit the model
grid_search.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search.best_params_)
print("Best CV ROC-AUC Score:", grid_search.best_score_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best Hyperparameters: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 200}
Best CV ROC-AUC Score: 0.7775014364769756


In [7]:
# Evaluate on test set
y_pred = grid_search.predict(X_test)
y_pred_proba = grid_search.predict_proba(X_test)[:, 1]

In [8]:
# Metrics
roc_auc_scores = cross_val_score(
    grid_search.best_estimator_, X_test, y_test,
    cv=5, scoring='roc_auc'
)

accuracy_scores = cross_val_score(
    grid_search.best_estimator_, X_test, y_test,
    cv=5, scoring='accuracy'
)


roc_auc_mean = np.mean(roc_auc_scores)
roc_auc_std = np.std(roc_auc_scores)

accuracy_mean = np.mean(accuracy_scores)
accuracy_std = np.std(accuracy_scores)


print("\nClassification Report:\n", classification_report(y_test, y_pred))

print(f"Test ROC-AUC: {roc_auc_mean:.4f} ± {roc_auc_std:.4f}")
print(f"Test Accuracy: {accuracy_mean:.4f} ± {accuracy_std:.4f}")


Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.98      0.89       802
           1       0.60      0.13      0.21       199

    accuracy                           0.81      1001
   macro avg       0.71      0.55      0.55      1001
weighted avg       0.78      0.81      0.76      1001

Test ROC-AUC: 0.7633 ± 0.0288
Test Accuracy: 0.8062 ± 0.0127
